# UrbanSoundsClassification: AIYard

Önişleme.ipynb

AIYard ekibi

### 1. Kütüphanelerin içe aktarımı

In [1]:
import cv2 as cv
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from random import shuffle

### 2. Spektrogramların okunması ve işlenmesi
Google Colaboratory'de spektrogram verilerini okuyup önişlemeye tabi tutarken hafıza limitini aştığımız için sıkıntı yaşadık.

Bunun önüne geçmek için spektrogramları verilen Drive linkinden indirdik ve lokal cihazlarımızda önişlemeye tabi tuttuk.

Alttaki kodda yapılacak tek değişiklik walk() metoduna verilecek olan klasörün yoludur.

In [2]:
data = [] # Okunan spektrogramları ve sınıflarının demetlerini tutacağımız liste
img_size = 64 # Spektrogramların küçültülecekleri boyut

for path, subdirs, files in os.walk("/home/ferit/repos/UrbanSoundsClassification/spectrograms/"):
    for name in files:
        file = os.path.join(path, name)
        spectrogram = cv.imread(file, 0) # Dosyayı grayscale olacak şekilde açmak için 0 pozisyonel argümanını veriyoruz
        spectrogram = cv.resize(spectrogram, (img_size, img_size), interpolation=cv.INTER_LINEAR) # Görüntüyü bilineer interpolasyonla küçültüyoruz 
        spectrogram = spectrogram.astype("float64") # Normalizasyonda yüksek doğruluklu ondalıklı sayılar elde etmek için matris veri tipini float64 olarak değiştiriyoruz 
        spectrogram /= 255 # Piksel değerleri maksimum 255 olacağı için tüm hücreleri 255'e bölerek değerleri 0 ile 1 arasına çekiyoruz
        data.append((spectrogram, int(file.split('/')[6]))) # data listesine spektrogram ve sınıfı içeren demeti ekliyoruz

In [3]:
print(len(data)) # Toplam (spektrogram, sınıf) demet sayısı 8735 olmalı

8735


### 3. "data" listesindeki demetleri rastgele karıştırıyoruz

In [4]:
print(f"Shuffle öncesi ilk elemanın sınıfı: {data[0][1]}")
shuffle(data)
print(f"Shuffle sonrası ilk elemanın sınıfı: {data[0][1]}")

Shuffle öncesi ilk elemanın sınıfı: 2
Shuffle sonrası ilk elemanın sınıfı: 7


### 4. Önişlenen görselleri preproc/set/spec-count-class.npy formatında kaydediyoruz
Modeli eğitmek için kullanılacak veriyi Model.ipynb dosyasında tekrardan elde etmeyi kolaylaştırmak için çıktı klasörü preproc'un altında

- train
- val
- test

klasörleri oluşturuyor ve spektrogramları "spec-numara-sınıf_sayısı.npy" formatında kaydediyoruz.

In [5]:
X_train = list(zip(*data[0:6900]))[0]
Y_train = list(zip(*data[0:6900]))[1]
X_val = list(zip(*data[6900:7800]))[0]
Y_val = list(zip(*data[6900:7800]))[1]
X_test = list(zip(*data[7800:8735]))[0]
Y_test = list(zip(*data[7800:8735]))[1]

In [6]:
try:
    os.mkdir("preproc")
    os.mkdir("preproc/train")
    os.mkdir("preproc/val")
    os.mkdir("preproc/test")
except FileExistsError:
    print("Klasör zaten oluşturulmuş")

#### 1. Train verilerini kaydet

In [7]:
index = 0
for spectrogram in X_train:
    np.save(f"preproc/train/spec-{index}-{Y_train[index]}.npy", spectrogram)
    index += 1

#### 2. Validasyon verilerini kaydet

In [8]:
index = 0
for spectrogram in X_val:
    np.save(f"preproc/val/spec-{index}-{Y_val[index]}.npy", spectrogram)
    index += 1

#### 3. Test verilerini kaydet

In [9]:
index = 0
for spectrogram in X_test:
    np.save(f"preproc/test/spec-{index}-{Y_test[index]}.npy", spectrogram)
    index += 1

### 5. Sonuç
Bütün hücreleri doğru girişleri sağlayarak (lokale indirilmiş spektrogram klasörü) çalıştırdığımızda repository kökünde `preproc` klasörü altında 3 klasör meydana gelmeli.

Bu klasörler:
- train (6900 adet spektrogram)
- val (900 adet spektrogram)
- test (935 adet spektrogram)
biçiminde olacaklar. Her klasör isim formatı `spec-dosya_numarası-sınıf_numarası.npy` biçiminde olan spektrogram dosyalarının NumPy array formatında kaydedilmiş versiyonunu içerecek.

Model.ipynb dosyasında klasörler sırası ile okunurken ismin `sınıf_numarası` kısmı ile spektrogramlar ile sınıfları eşlenerek demetlere kaydedilerek kullanılacak.